In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import plotly.express as px

from statsmodels.stats.weightstats import ttest_ind

from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import LogisticRegression
from statsmodels.api import OLS, add_constant
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

from statsmodels.api import Logit
from sklearn.metrics import confusion_matrix


Import the two csv with the data

In [2]:
bretagne = pd.read_csv('Bretagne.csv')

In [3]:
idf = pd.read_csv('Paris.csv')

In [4]:
# create unique file (concatenate)

df = pd.concat([idf, bretagne])

In [6]:
# clean the file 

df.drop(['URL source','ID de la transaction','Montant du paiement','Date du paiement','État du paiement','ID de l’article','Merci pour votre implication, laissez-nous votre adresse mail pour être recontacté'],axis=1, inplace=True)

In [7]:
# export the file as csv
#since we have done changes on the csv directly we comment this part to avoid overwrting the csv

df.to_csv('Full_Question_vaccin_V1.1', index=False)

In [8]:
# read the file 

df1=pd.read_csv('Full_Question_vaccin_V1.1.csv')

FileNotFoundError: [Errno 2] File Full_Question_vaccin_V1.1.csv does not exist: 'Full_Question_vaccin_V1.1.csv'

In [ ]:
# creating a dictionary to map departments number and region name
region = {56 : 'Bretagne', 29 :'Bretagne',35 : 'Bretagne', 22 : 'Bretagne', 75 : 'IDF', 78 : 'IDF', 92 : 'IDF',91:'IDF', 77 : 'IDF', 94 : 'IDF', 95 : 'IDF', 93 :'IDF'}

In [ ]:
# create the region column by mapping the dictionary above

df1['region']= df1.Dept.map(region)

In [ ]:
# create the age column

df['Age'] = 2020-df['Birth']

In [ ]:
# define the category age

def category_age(age):
    if age <= 30:
        return 0
    elif (age>30) & (age<=45):
        return 1
    elif (age>45) & (age<=60):
        return 2
    else : return 3

In [ ]:
# create the cat_age column by mapping the function above

df['cat_age'] = df.Age.map(category_age)


In [ ]:
# distribution by sex

df_test = df1.groupby('Sexe').size()
df_test=pd.DataFrame({"count":df_test})

fig= px.bar(df_test.reset_index(), x='Sexe',y='count',color='Sexe',color_discrete_sequence=['pink','blue'],text='count')
fig.update_traces(textposition='inside')
fig.show()

In [ ]:
# distribution by sex and region

df_test1 = df1.groupby(['region','Sexe']).count().reset_index()
df_test1.rename(columns={'Dept':'Count'},inplace=True)

fig = px.bar(df_test1,x='region',y='Count',color='Sexe',color_discrete_sequence=['pink','blue'])
fig.show()

In [ ]:
## Number of Y/N by Catergory

df_test5 = df1.groupby(['Profession','Did_last_vacc_campaign']).size().sort_values(ascending=False).reset_index()
df_test5.rename(columns={0:'Count'},inplace=True)
fig = px.bar(df_test5,y='Profession',x='Count',color='Did_last_vacc_campaign',color_discrete_sequence=['green','red'],text='Count',title='Number Y/N by Category')
fig.update_traces(textposition='inside')
fig.show()

In [ ]:
# age ditribution

df1['Age'].hist()

In [ ]:
df_test7 = df1.groupby(['cat_age','Did_last_vacc_campaign']).size().sort_values(ascending=False).reset_index()
df_test7.rename(columns={0:'Count'},inplace=True)
fig = px.bar(df_test7,x='cat_age',y='Count',color='Did_last_vacc_campaign',color_discrete_sequence=['green','red'],
             text='Count',title='Number Y/N by cat_age')
fig.update_traces(textposition='inside')
fig.show()

In [ ]:
# Statistic about 'Proud odf Profession'

df['Proud_profession_score']=df['Proud_profession'].isin(["D\'accord",'Tout à fait d\'accord']).map({True:"Oui",False:'Non'})
pd.crosstab(df['Did_last_vacc_campaign'],df['Proud_profession_score'],normalize='columns', margins=True)

prouds = df.query('Proud_profession_score=="Oui"')['Did_last_vacc_campaign'].map({'Oui':1,'Non':0})
nonprouds = df.query('Proud_profession_score=="Non"')['Did_last_vacc_campaign'].map({'Oui':1,'Non':0})

ttest_ind(prouds,nonprouds)

In [ ]:
# Statistic about 'Profession Values Identification'

df['Profession_values_identification_score']=df['Profession_values_identification'].isin(["D\'accord",'Tout à fait d\'accord']).map({True:"Oui",False:'Non'})

pd.crosstab(df['Did_last_vacc_campaign'],df['Profession_values_identification_score'],normalize='columns', margins=True)

identify = df.query('Profession_values_identification_score=="Oui"')['Did_last_vacc_campaign'].map({'Oui':1,'Non':0})
nonidentify = df.query('Profession_values_identification_score=="Non"')['Did_last_vacc_campaign'].map({'Oui':1,'Non':0})

ttest_ind(identify,nonidentify)

In [ ]:
# Statistic about 'Cohesion Feeling'

df['Profession_high_cohesion_feeling_score']=df['Profession_high_cohesion_feeling'].isin(["D\'accord",'Tout à fait d\'accord']).map({True:"Oui",False:'Non'})

pd.crosstab(df['Did_last_vacc_campaign'],df['Profession_high_cohesion_feeling'],normalize='columns', margins=True)

cohesion = df.query('Profession_high_cohesion_feeling_score=="Oui"')['Did_last_vacc_campaign'].map({'Oui':1,'Non':0})
noncohesion = df.query('Profession_high_cohesion_feeling_score=="Non"')['Did_last_vacc_campaign'].map({'Oui':1,'Non':0})

ttest_ind(cohesion,noncohesion)

In [ ]:
# Going deeper in people saying No at last vaccination campaign
# Create DataFrame about this people

df_no = df.loc[df['Did_last_vacc_campaign']=='Non'].copy()

In [ ]:
# test all professions who didnt do the last vaccin, if they will do the next one

from scipy.stats import f_oneway

ddf={}
for i,j in df.groupby('Profession'):
    ddf[i]=j.query('Did_last_vacc_campaign=="Non"').Will_vacc_next_campaign_2.map({'Oui':1,'Non':0})
ddf.keys()

In [ ]:
# Anova test

f_oneway(*ddf.values())

In [ ]:
# Proportion of people will do the next vaccin campaign by profession 

dict(map(lambda x:(x[0],x[1].mean()), ddf.items()))

In [ ]:
# Sample size of the previous calculation

dict(map(lambda x:(x[0],x[1].size), ddf.items()))

In [ ]:
# Verify or Unverify the Witte Theory : 

## People said they will not do the next vaccination campaign, score should be < 0 
## People said they will do the next vaccination campaign, score should be > 0 

negative=df.loc[df['Difference des 2 blocs']<1, 'Will_vacc_next_campaign_2'].map({'Non':0,'Oui':1}).dropna()
positive=df.loc[df['Difference des 2 blocs']>=1, 'Will_vacc_next_campaign_2'].map({'Non':0,'Oui':1}).dropna()

ttest_ind(negative,positive)

In [ ]:
df['score']=df['Difference des 2 blocs'].map(lambda x: 'above 1' if x>=1 else 'below 1' if x<1 else np.nan)
df['answer']=df['Will_vacc_next_campaign_2'].map({'Non':0,'Oui':1})

In [ ]:
df.groupby('score')['answer'].mean().round(2)

Based on descriptive statistic If the score is above 1, probability he will get vaccinated are 51
The higher score the higher the chances.
Since for the other 50% the theory is not confirmed let’s go deeper

In [ ]:
# BUilding Decision Tree Model on Witte Theory

df_ml=df_no[['Will_vacc_next_campaign_2','Difference des 2 blocs']].copy()
df_ml.reset_index(inplace=True)
df_ml.drop('index',axis=1,inplace=True)
df_ml['Will_vacc_next_campaign_2']=df_ml['Will_vacc_next_campaign_2'].map({'Non':0,'Oui':1})

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(
df_ml.drop('Will_vacc_next_campaign_2',axis=1),
df_ml['Will_vacc_next_campaign_2'],
test_size=1/3,
random_state=42,
stratify = df_ml['Will_vacc_next_campaign_2']
)

In [ ]:
# 1st try on Decision Tree

dt=DecisionTreeClassifier(class_weight='balanced')
dt.fit(X_train, y_train)

In [ ]:
# Final try on decision

clf = DecisionTreeClassifier(max_leaf_nodes=5, random_state=0, min_samples_split=10)
clf.fit(X_train, y_train)

In [ ]:
#plot the decision tree 'clf'

plt.figure(figsize=(50,25))
tree.plot_tree(clf)
plt.show()

In [ ]:
# add color to the chart

fn=['Difference des 2 blocs']
cn=['Non', 'Oui']
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (4,4), dpi=300)
tree.plot_tree(clf,
               feature_names = fn, 
               class_names=cn,
               filled = True);

In [ ]:
# Going deeper on the score analysis for the Witte Theory

# function to define category

def score_cat(score):
    if score<=-11.5:
        return 'below -11.5'
    elif score<=-2.5:
        return '-11.5 to -2.5'
    elif score <=8.5:
        return '-2.5 to 8.5'
    elif score <=14.5:
        return '8.5 to 14.5'
    else:
        return 'above 14.5'

In [ ]:
# Map the function to the score to have a new function

df['difference des 2 blocs_score']=df['Difference des 2 blocs'].map(score_cat)
df['will_do']=df['Will_vacc_next_campaign_2'].map({'Non':0, 'Oui':1})

In [ ]:
df.rename(columns={'difference des 2 blocs_score':'score'},inplace=True)

In [ ]:
# Building a heatmap of % per category score

sns.heatmap(df[['score', 'will_do']].groupby('score').apply(lambda x: x.will_do.mean()).loc[['above 14.5','8.5 to 14.5','-2.5 to 8.5','-11.5 to -2.5','below -11.5' ]].to_frame(),annot=True)
plt.xlabel('% Yes Respondents')

In [ ]:
# Covid impact on decision to do the vaccin

df_no.groupby(['Will_vacc_next_campaign_2','covid_context_influence_score']).size().reset_index()

In [ ]:
# Create the modeling part

In [ ]:
# Create X & Y 

X = df_no.drop(['Will_vacc_next_campaign_2'],axis=1)
y = df_no['Will_vacc_next_campaign_2'].map({'Oui':1,'Non':0})

In [ ]:
# Create categories from profession

X['profession']=X['Profession'].astype('category').cat.codes

In [ ]:
# Cleaning missing data on X (Need some review - some columns still can be usefull)

X = X.loc[:, X.isnull().mean() < .55]

In [ ]:
# Cleaning unuseful columns and too correlated one

X['easy_acc']=X['easy_acc'].map({'Oui':1,'Non':0})
X['region'] =X.region.map({'IDF':1,'Bretagne':2})
X.drop(['Proud_profession_score','Profession_values_identification_score','Profession','Did_last_vacc_campaign',
        'Les étudiants craignent d’être marginalisés s’ils se font vacciner alors que les professionnels qui les entourent ne le font pas',
       'easy_acc','Profession','Birth','Dept',],axis=1,inplace=True)
X.Sexe = X.Sexe.map({'Homme':0,'Femme':1})

In [ ]:
# Dictionnary to map text score into ML useful data

score = {'Pas du tout d\'accord':-3,
         'Pas d\'accord':-2,
         'Plutôt pas d\'accord':-1,
         'Indifférent':0,
         'Plutôt d\'accord':1,
         'D\'accord':2,
         'Tout à fait d\'accord':3}

In [ ]:
# Map the dictionnary

X[['Proud_profession','Profession_values_identification','Profession_define_myself','Profession_high_cohesion_feeling','Professional_group_being_part','As_ProfessionalH_MustBe_Vacc','Vac_forProfessionalH_Choice','Accept_Vac_Proof_Resp','Choice_Vacc_limited_by_patient_protect']]=X[['Proud_profession','Profession_values_identification','Profession_define_myself','Profession_high_cohesion_feeling','Professional_group_being_part','As_ProfessionalH_MustBe_Vacc','Vac_forProfessionalH_Choice','Accept_Vac_Proof_Resp','Choice_Vacc_limited_by_patient_protect']].apply(lambda x: x.map(score), axis=1)

In [ ]:
# Fill the last missing value

X.fillna(value=0,inplace=True)

In [ ]:
# Visualise correlation

sns.heatmap(X.corr())

In [ ]:
# Select the best features with SFS

estimator=LogisticRegression(max_iter=1e8)
sfs1=sfs(
    estimator,
    k_features=10,
    forward=True,
    floating=False,
    scoring='accuracy'
)

In [ ]:
sfs1.fit(X,y)

In [ ]:
sfs1.get_metric_dict()

In [ ]:
# Redefine X with the result of SFS

X=sfs1.transform(X)

In [ ]:
# Building a Logistic Regression Model

Logit(y.reset_index(drop=True), X).fit(maxiter=1e8).summary()

In [ ]:
# Base on result we drop column with P value > 0.05

X.drop('Difference des 2 blocs',axis=1,inplace=True)

Logit(y.reset_index(drop=True), X).fit(maxiter=1e8).summary()

In [ ]:
X.drop('Cumul 1er bloc',axis=1,inplace=True)

Logit(y.reset_index(drop=True), X).fit(maxiter=1e8).summary()

In [ ]:
# Print the result of the model

model=Logit(y.reset_index(drop=True), X).fit()
model.summary()

In [ ]:
# Cohesion matrix

proba_pred=model.predict()
proba_pred=np.where(proba_pred>0.5,1, 0)

confusion_matrix(y, proba_pred)